# 3D Interpolation with Scipy

The hall probe measurement simulation software is hamstrung by the sparseness of the available field grid data.  The Mau10 and GA05 field grids are given in Cartesian coordinates, with 25 mm spacing on the X, Y, and Z axes.  The hall probe measurement device will work most naturally in cylindrical coordinates.  In order to sample regular cylindrically symmetric data from the field grids, one must either cherry pick values that coincide with the 25x25x25 mm spacing, or generate data through interpolation.

The current interpolation method employed in the Mu2E organization for doing this is a linear 3D method.  This leads to can lead to fairly large errors in regions of large magnetic gradient, which would be unsuitable for further analysis.  We will attempt to use a more robust interpolate method and compare those results with random field data (provided in a separate file for validation purposes).

In [1]:
from mu2e.dataframeprod import DataFrameMaker
from mu2e import mu2e_ext_path
import numpy as np

Welcome to ROOTaaS 6.06/06


In [2]:
df = DataFrameMaker(mu2e_ext_path+'datafiles/Mau10/Standard_Maps/Mu2e_DSMap',use_pickle = True).data_frame

In [4]:
df.head()

,X,Y,Z,Bx,By,Bz,R,Phi,Bphi,Br
0,-1200.0,-1200.0,3071.0,0.129282,0.132041,0.044331,1697.056275,-2.356194,-0.001951,-0.184783
1,-1200.0,-1200.0,3096.0,0.132108,0.134881,0.041163,1697.056275,-2.356194,-0.001960,-0.188790
2,-1200.0,-1200.0,3121.0,0.134887,0.137672,0.037731,1697.056275,-2.356194,-0.001969,-0.192728
3,-1200.0,-1200.0,3146.0,0.137602,0.140399,0.034029,1697.056275,-2.356194,-0.001977,-0.196576
4,-1200.0,-1200.0,3171.0,0.140237,0.143045,0.030050,1697.056275,-2.356194,-0.001985,-0.200311


In [3]:
df_rand = DataFrameMaker(mu2e_ext_path+'datafiles/Mau10/Standard_Maps/Mu2e_DSMap_rand1mil',use_pickle = True).data_frame

In [6]:
df_rand.head()

,X,Y,Z,Bx,By,Bz,R,Phi,Bphi,Br
0,-1007.9996,-0.708404,12190.6920,-0.005847,-0.000004,0.993192,1007.999849,-3.140890,-1.790000e-07,0.005847
1,-1007.9966,-1.116473,7536.1132,-0.564179,-0.000629,1.455297,1007.997218,-3.140485,4.159000e-06,0.564180
2,-1007.9943,-1.397842,5638.5763,0.147173,0.000207,1.172049,1007.995269,-3.140206,-3.246000e-06,-0.147173
3,-1007.9935,-1.022141,7750.8171,-0.221387,-0.000226,0.703601,1007.994018,-3.140579,1.061000e-06,0.221387
4,-1007.9879,-1.502013,8048.6632,0.059701,0.000091,1.010066,1007.989019,-3.140103,-1.725000e-06,-0.059701


In [7]:
df_rand.Y.max()

999.95324000000005

In [4]:
df_rand_subset = df_rand.query('-800<=X<=800 and -800<=Y<=800 and 4200<=Z<=13000')

We have our dataframes queued up.  We will attempt to interpolate a regular grid from `df`, and test values from `df_rand` to determine our accuracy.  Let's try to use scipy.interpolate.Rbf in a simple case, first:

In [5]:
from scipy.interpolate import Rbf

In [6]:
x, y, z, d = np.random.rand(4, 50)
rbfi = Rbf(x, y, z, d)
xi = yi = zi = np.linspace(0, 1, 20)
di = rbfi(xi, yi, zi)

In [7]:
print di

[ 0.09790102  0.15664449  0.2462196   0.381434    0.56512296  0.77834237
  0.97383241  1.07885549  1.04508479  0.91029727  0.76698539  0.64418499
  0.46331124  0.20230294 -0.00359683 -0.04662047  0.04660403  0.19832145
  0.35334717  0.48787137]


In [8]:
def cylindrical_norm(x1,x2):
    return np.sqrt(
        (x1[0,:]*np.cos(x1[1,:])-x2[0,:]*np.cos(x2[1,:]))**2 +
        (x1[0,:]*np.sin(x1[1,:])-x2[0,:]*np.sin(x2[1,:]))**2 +
        (x1[2,:]-x2[2,:])**2)

In [9]:
my_rand = df_rand_subset.sample(1)

In [10]:
my_rand.X.values

array([-577.5064])

In [11]:
df_subset = df.query('{0}<=X<={1} and {2}<=Y<={3} and {4}<=Z<={5}'.format(
    my_rand.X.values-50, my_rand.X.values+50, my_rand.Y.values-50, my_rand.Y.values+50, my_rand.Z.values-50, my_rand.Z.values+50))

In [12]:
x = df_subset.X
y = df_subset.Y
z = df_subset.Z
r = df_subset.R
phi = df_subset.Phi
bz = df_subset.Bz
br = df_subset.Br
bphi = df_subset.Bphi



In [18]:
rbf_bz = Rbf(r, phi, z, bz,function='linear', norm=cylindrical_norm)
rbf_br = Rbf(r, phi, z, br,function='linear', norm=cylindrical_norm)

In [14]:
i_bz = rbf_bz(my_rand.R, my_rand.Phi, my_rand.Z)
t_bz = my_rand.Bz

In [15]:
res =abs(i_bz-t_bz)

In [16]:
max(res)

7.6403455988138802e-05

In [23]:
buffer = 100
counter = 0
for i in xrange(100):
#for i in xrange(13,14):
    my_rand = df_rand_subset.sample(1,random_state=i+10)
    df_subset = df.query('{0}<=X<={1} and {2}<=Y<={3} and {4}<=Z<={5}'.format(
        my_rand.X.values-buffer, my_rand.X.values+buffer, my_rand.Y.values-buffer, my_rand.Y.values+buffer, my_rand.Z.values-buffer, my_rand.Z.values+buffer))
    x = df_subset.X
    y = df_subset.Y
    z = df_subset.Z
    r = df_subset.R
    phi = df_subset.Phi
    bz = df_subset.Bz
    br = df_subset.Br
    bphi = df_subset.Bphi
    rbf_br = Rbf(r, phi, z, br,function='linear', norm=cylindrical_norm, epsilon=20)
    i_br = rbf_br(my_rand.R, my_rand.Phi, my_rand.Z)
    t_br = my_rand.Br
    res = abs(i_br-t_br).values
    if res>1e-4:
        print res[0], i
        print my_rand
        print '',i_br[0]
        counter +=1
    else: print res
print 'bads:', counter

[  6.55961570e-06]
[  1.35325646e-05]
[  9.54488972e-06]
[  1.61405731e-05]
[  1.61176621e-05]
[  1.46837251e-05]
[  1.69284015e-05]
[  3.84124519e-05]
[  1.77232320e-05]
[  2.09387690e-06]
[  1.48291022e-05]
[  4.90008976e-05]
[  4.22863263e-06]
[  5.50250170e-05]
[  2.09724114e-05]
[  1.31417269e-05]
[  7.46238260e-06]
[  1.20105919e-05]
[  1.00106109e-05]
[  6.59056459e-06]
[  2.32124953e-06]
[  2.04700809e-06]
[  5.38651632e-06]
[  2.52862800e-05]
[  2.00886663e-05]
[  2.46545714e-05]
[  2.11654759e-05]
[  5.84377312e-06]
[  2.62918198e-05]
[  9.21297218e-06]
[  2.84631281e-05]
[  1.71382513e-05]
[  3.47427623e-06]
[  2.95162789e-06]
[  5.28918301e-05]
[  3.36116119e-06]
[  4.86566465e-06]
[  2.29497200e-05]
[  4.42148448e-05]
[  1.82738924e-05]
[  3.49650993e-06]
[  1.09377174e-05]
[  1.85129528e-05]
[  2.44935131e-05]
[  1.92260382e-05]
[  3.26790220e-05]
[  1.26690171e-05]
[  4.84334218e-06]
[  2.45729161e-05]
[  1.63687039e-05]
[  1.55581293e-05]
[  1.94867640e-05]
[  3.6279186

In [ ]:
print np.asarray(res_lin)-np.asarray(res_quin)

In [ ]:
rbf_br = Rbf(r, phi, z, br,function='gaussian', norm=cylindrical_norm)
i_br = rbf_br(df_rand_subset.R, df_rand_subset.Phi, df_rand_subset.Z)
t_br = df_rand_subset.Br
res =abs(i_br-t_br)
max(res)

In [ ]:
rbf_bphi = Rbf(r, phi, z, bphi,function='quintic', norm=cylindrical_norm)
i_bphi = rbf_bphi(df_rand_subset.R, df_rand_subset.Phi, df_rand_subset.Z)
t_bphi = df_rand_subset.Bphi
res =abs(i_bphi-t_bphi)
max(res)